In [1]:
import tweepy as tw
import pandas as pd
import re
import glob
import os
import schedule
import time
from datetime import datetime

%run ./fair_keys.ipynb

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [2]:
#keys = ['Spy x Family','Genshin Impact']
keys = ['BL','anime comedy','anime romance','school life','anime','animation','Shounen','Pixar',
        'harem','fantasy anime','sport anime','from manga','from novel','animation studio',
        'shounen ai','seinen','shoujo','slice of life anime']

In [3]:
def savedata():
    def get_related_tweets(key_word):
        tweet_keyword = []
        twitter_users = []
        #twitter_users_id = []
        tweet_time = []
        tweet_string = [] 
        tweet_countRT = []
        tweet_fav = []
        for tweet in tw.Cursor(api.search_tweets,q=key_word,tweet_mode="extended",include_entities=True).items(40):
                if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
                    if tweet.lang == "en":
                        twitter_users.append(tweet.user.screen_name)
                        #twitter_users_id.append(tweet.user.id)
                        tweet_time.append(tweet.created_at)
                        tweet_string.append(tweet.full_text)
                        tweet_countRT.append(tweet.retweet_count)
                        tweet_fav.append(tweet.favorite_count)
                        tweet_keyword.append(key_word)
        df = pd.DataFrame({'Keyword':tweet_keyword,'User':twitter_users,'Retweet':tweet_countRT,'Likes':tweet_fav, 'Tweet': tweet_string, 'Time': tweet_time})
        #df.to_csv(f"{key_word}.csv")
        return df

    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    filename = "tweet_data.csv"

    if filename not in glob.glob("*.csv"):
        df = pd.DataFrame(columns=['Keyword','User','Retweet','Likes','Tweet','Time'])
    else:
        df = pd.read_csv(filename)
        os.remove('tweet_data.csv')

    for keyword in keys:
        df = pd.concat([df,get_related_tweets(keyword)])

    #df['Time'] = pd.to_datetime(df['Time'])
    #df['Likes'] = df['Likes'].astype(str).astype(int)
    #df['Retweet'] = df['Retweet'].astype(str).astype(int)
    df.drop_duplicates(keep='last',inplace=True)
    df.sort_values(by=['Keyword'],inplace=True)
    df.to_csv(filename,encoding='utf-8',index=False)
    print('save complete',current_time)

In [4]:
savedata()

schedule.every(15).minutes.do(savedata)
while True:
    schedule.run_pending()
    time.sleep(1)

save complete 15:38:34
save complete 15:54:00
save complete 16:09:26
save complete 16:24:52
save complete 16:40:18


KeyboardInterrupt: 

In [ ]:
#df.loc[df['Keyword']=='from manga']

In [ ]:
#df.info()

In [ ]:
#df